## Apress - Industrialized Machine Learning Examples

Andreas Francois Vermeulen
2019

### This is an example add-on to a book and needs to be accepted as part of that copyright.

## Chapter-010-02-Distributed-Evolutionary-Algorithms

Distributed Evolutionary Algorithms

Install the library:

In [1]:
#conda install -c conda-forge deap

## Part A - Import the libraries:

In [2]:
import operator
import random
import numpy
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

## Part B - Set Creator's Fitness Maximum

In [3]:
creator.create("FitnessMax", 
               base.Fitness, 
               weights=(1.0,)
              )

## Part C - Activate Creator's Particle List

In [4]:
creator.create("Particle", 
               list, 
               fitness=creator.FitnessMax, 
               speed=list, 
               smin=None, 
               smax=None, 
               best=None
              )

## Part D - Create function to create a particle

In [5]:
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

## Part E - Create function to update particle

In [6]:
def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))


## Part F - Setup the Toolbox

In [7]:
toolbox = base.Toolbox()

toolbox.register("particle", 
                 generate, 
                 size=2, 
                 pmin=-6, 
                 pmax=6, 
                 smin=-3, 
                 smax=3
                )

toolbox.register("population", 
                 tools.initRepeat, 
                 list, 
                 toolbox.particle
                )

toolbox.register("update", 
                 updateParticle, 
                 phi1=2.0, 
                 phi2=2.0
                )

toolbox.register("evaluate", 
                 benchmarks.h1
                )

## Part G - Setup Main Function

In [8]:
def main_proc():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        #print(logbook.stream)
    
    return pop, logbook, best

## Part H - Run the Solution

In [9]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [10]:
pop, logbook, best = main_proc()

In [11]:
print('### Population #########################################')
print(pop)

### Population #########################################
[[8.859222017042846, 8.86081353071928], [7.034872043176576, 5.673874271607648], [8.302874877300217, 5.855909670643183], [8.451983067539045, 5.65371001334071], [8.636251418048257, 5.261946400084986]]


In [12]:
print('### Log Book ###########################################')
print(logbook)

### Log Book ###########################################
gen	evals	avg      	std      	min        	max      
0  	5    	0.0550787	0.0383715	0.0160405  	0.120579 
1  	5    	0.0527326	0.0341327	0.0125371  	0.11388  
2  	5    	0.0990021	0.0216002	0.0759578  	0.127383 
3  	5    	0.0618895	0.0372504	0.00488041 	0.122463 
4  	5    	0.0574621	0.0351342	0.014704   	0.103573 
5  	5    	0.0717995	0.0376846	0.00873124 	0.123033 
6  	5    	0.100522 	0.0362791	0.0447606  	0.158202 
7  	5    	0.0673803	0.0284525	0.0168088  	0.0978442
8  	5    	0.076034 	0.0511039	0.0187997  	0.147621 
9  	5    	0.0851316	0.0392099	0.0331847  	0.154933 
10 	5    	0.0907236	0.0428325	0.0596286  	0.174231 
11 	5    	0.12799  	0.0374589	0.0732687  	0.167085 
12 	5    	0.229078 	0.11153  	0.124298   	0.445593 
13 	5    	0.276547 	0.114587 	0.123676   	0.460745 
14 	5    	0.179743 	0.0989254	0.0775324  	0.363717 
15 	5    	0.232952 	0.0732469	0.131387   	0.352508 
16 	5    	0.187498 	0.114928 	0.0248392  	0.376251 
17 	5  

In [13]:
print('### Best ###############################################')
print(best)

### Best ###############################################
[8.674390945206916, 6.781872632623891]


## Done

In [14]:
import datetime
now = datetime.datetime.now()
print('Done!',str(now))

Done! 2019-04-27 12:30:09.363246
